In [10]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

def load_train_data(train_path, train_batch_size, size, shuffle = True):
    transformers = transforms.Compose([
                    transforms.Resize((size, size)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    set_data = datasets.ImageFolder(root = train_path, transform = transformers)
    return DataLoader(set_data, batch_size = train_batch_size, shuffle = shuffle, num_workers = 1)

def load_test_data(test_path, test_batch_size, size, shuffle = True):
    transformers = transforms.Compose([transforms.Resize((size, size)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    set_data = datasets.ImageFolder(root = test_path, transform = transformers)
    return DataLoader(set_data, batch_size = test_batch_size, shuffle = shuffle, num_workers = 1)

In [11]:
import torch.nn as nn


class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16,
                               kernel_size=3, padding=1)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32,
                               kernel_size=3, padding=1)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64,
                               kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64,
                               kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2)

        self.dropout = nn.Dropout()

        self.fc1 = nn.Linear(in_features=1024, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

        self.relu = nn.ReLU()


    def forward(self, x):
        #64x64x3
        x = self.relu(self.conv1(x))
        #64x64x16
        x = self.maxpool(x)
        #32x32x16
        x = self.relu(self.conv2(x))
        #32x32x32
        x = self.maxpool(x)
        #16x16x32
        x = self.relu(self.conv3(x))
        #16x16x64
        x = self.maxpool(x)
        #8x8x64
        x = self.relu(self.conv4(x))
        #8x8x64
        x = self.maxpool(x)
        #4x4x64
        #Flatening
        x = x.view(-1, 1024)

        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x 

In [18]:
import torch
# from Models import ConvNet
# from Load import load_data

from tqdm import tqdm


EPOCHS = 25
Learning_rate = 0.001
Batch_size = 64
L2_rate = 0

image_size = 64
data_size = 9017
num_batches = data_size//Batch_size
num_classes = 7

Train_path = "C:/Users/Home/Desktop/Mnist_/skin-cancer-mnist-ham10000/data/train/"


model = ConvNet(num_classes)


data_loader = load_train_data(Train_path, Batch_size, image_size)


def train():
    model.train()

    crossentropy = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=Learning_rate)

    for epoch in range(EPOCHS):
        epoch_loss = 0
        epoch_acc = 0
        for X, y in tqdm(data_loader):

            optimizer.zero_grad()

            out = model(X) 

            
            loss = crossentropy(out, y)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item() 

            predictions = torch.argmax(out, 1)
            epoch_acc += torch.sum(predictions==y).item()

        epoch_loss = epoch_loss/num_batches
        epoch_acc = epoch_acc/data_size
        print(f"Epoch {epoch}:")
        print("ACC:", epoch_acc, "LOSS:", epoch_loss)

        torch.save(model.state_dict(), f"C:/Users/Home/Desktop/Mnist_/skin-cancer-mnist-ham10000/saved_models/ConvNet_{epoch}.model")


In [19]:
train()



  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

  1%|▌                                                                                 | 1/141 [00:02<05:26,  2.33s/it]

  1%|█▏                                                                                | 2/141 [00:03<04:28,  1.93s/it]

  2%|█▋                                                                                | 3/141 [00:04<03:57,  1.72s/it]

  3%|██▎                                                                               | 4/141 [00:05<03:28,  1.52s/it]

  4%|██▉                                                                               | 5/141 [00:06<03:08,  1.39s/it]

  4%|███▍                                                                              | 6/141 [00:07<02:51,  1.27s/it]

  5%|████                                                                              | 7/141 [00:08<02:41,  1.20s/it]

  6%|████▋                    

 95%|████████████████████████████████████████████████████████████████████████████    | 134/141 [02:17<00:07,  1.03s/it]

 96%|████████████████████████████████████████████████████████████████████████████▌   | 135/141 [02:18<00:06,  1.07s/it]

 96%|█████████████████████████████████████████████████████████████████████████████▏  | 136/141 [02:19<00:05,  1.09s/it]

 97%|█████████████████████████████████████████████████████████████████████████████▋  | 137/141 [02:20<00:04,  1.05s/it]

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 138/141 [02:21<00:03,  1.04s/it]

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 139/141 [02:22<00:02,  1.02s/it]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 140/141 [02:23<00:01,  1.01s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [02:24<00:00,  1.05it/s]



Epoch 0:
ACC: 0.6656315847842963 LOSS: 1.0462121852806636




  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

  1%|▌                                                                                 | 1/141 [00:02<05:18,  2.27s/it]

  1%|█▏                                                                                | 2/141 [00:03<04:22,  1.89s/it]

  2%|█▋                                                                                | 3/141 [00:04<03:43,  1.62s/it]

  3%|██▎                                                                               | 4/141 [00:05<03:14,  1.42s/it]

  4%|██▉                                                                               | 5/141 [00:06<02:54,  1.29s/it]

  4%|███▍                                                                              | 6/141 [00:07<02:41,  1.20s/it]

  5%|████                                                                              | 7/141 [00:08<02:32,  1.14s/it]

  6%|████▋                    

 95%|████████████████████████████████████████████████████████████████████████████    | 134/141 [02:21<00:07,  1.02s/it]

 96%|████████████████████████████████████████████████████████████████████████████▌   | 135/141 [02:22<00:06,  1.05s/it]

 96%|█████████████████████████████████████████████████████████████████████████████▏  | 136/141 [02:23<00:05,  1.04s/it]

 97%|█████████████████████████████████████████████████████████████████████████████▋  | 137/141 [02:24<00:04,  1.05s/it]

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 138/141 [02:25<00:03,  1.04s/it]

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 139/141 [02:26<00:02,  1.04s/it]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 140/141 [02:27<00:01,  1.02s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [02:28<00:00,  1.01it/s]



...

 95%|████████████████████████████████████████████████████████████████████████████    | 134/141 [03:35<00:35,  5.11s/it]

 96%|████████████████████████████████████████████████████████████████████████████▌   | 135/141 [03:36<00:23,  4.00s/it]

 96%|█████████████████████████████████████████████████████████████████████████████▏  | 136/141 [03:37<00:15,  3.19s/it]

 97%|█████████████████████████████████████████████████████████████████████████████▋  | 137/141 [03:39<00:10,  2.62s/it]

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 138/141 [03:40<00:06,  2.18s/it]

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 139/141 [03:41<00:03,  1.91s/it]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 140/141 [03:42<00:01,  1.71s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [03:43<00:00,  1.48s/it]



Epoch 23:
ACC: 0.811245425307752 LOSS: 0.5053950386387961




  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

  1%|▌                                                                                 | 1/141 [00:02<05:42,  2.45s/it]

  1%|█▏                                                                                | 2/141 [00:03<04:43,  2.04s/it]

  2%|█▋                                                                                | 3/141 [00:04<04:04,  1.77s/it]

  3%|██▎                                                                               | 4/141 [00:05<03:34,  1.56s/it]

  4%|██▉                                                                               | 5/141 [00:06<03:10,  1.40s/it]

  4%|███▍                                                                              | 6/141 [00:07<02:53,  1.29s/it]

  5%|████                                                                              | 7/141 [00:08<02:43,  1.22s/it]

  6%|████▋                    

 95%|████████████████████████████████████████████████████████████████████████████    | 134/141 [03:10<00:06,  1.07it/s]

 96%|████████████████████████████████████████████████████████████████████████████▌   | 135/141 [03:11<00:05,  1.07it/s]

 96%|█████████████████████████████████████████████████████████████████████████████▏  | 136/141 [03:12<00:04,  1.07it/s]

 97%|█████████████████████████████████████████████████████████████████████████████▋  | 137/141 [03:13<00:03,  1.08it/s]

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 138/141 [03:14<00:02,  1.07it/s]

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 139/141 [03:15<00:01,  1.06it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 140/141 [03:16<00:00,  1.07it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [03:17<00:00,  1.14it/s]



Epoch 24:
ACC: 0.8156814905179106 LOSS: 0.48758397421666555


In [20]:
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm

model = ConvNet(7)

state_dict = torch.load("C:/Users/Home/Desktop/Mnist_/skin-cancer-mnist-ham10000/saved_models/ConvNet_24.model")
model.load_state_dict(state_dict)

data_size = 998
test_batch_size = 64
size = 64

test_path = "C:/Users/Home/Desktop/Mnist_/skin-cancer-mnist-ham10000/data/test/"

data_loader = load_test_data(test_path, test_batch_size, size, shuffle=False)

def test():
    model.eval()
    acc = 0
    y_hat = []
    y_true = []
    for X, y in tqdm(data_loader):
        
        out = model(X)
        
        predictions = torch.argmax(out, 1)
        acc += torch.sum(predictions == y).item()
        y_hat.append(predictions)
        y_true.append(y)
        
    y_hat = torch.cat(y_hat)
    y_true = torch.cat(y_true)
    acc = acc/data_size
    print(acc)
    print(classification_report(y_hat, y_true))

In [21]:
test()



  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

  6%|█████▏                                                                             | 1/16 [00:01<00:25,  1.69s/it]

 12%|██████████▍                                                                        | 2/16 [00:02<00:20,  1.45s/it]

 19%|███████████████▌                                                                   | 3/16 [00:03<00:16,  1.30s/it]

 25%|████████████████████▊                                                              | 4/16 [00:04<00:14,  1.17s/it]

 31%|█████████████████████████▉                                                         | 5/16 [00:05<00:11,  1.06s/it]

 38%|███████████████████████████████▏                                                   | 6/16 [00:06<00:09,  1.01it/s]

 44%|████████████████████████████████████▎                                              | 7/16 [00:06<00:08,  1.06it/s]

 50%|█████████████████████████

0.7635270541082164


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

          0       0.31      0.59      0.41        17
          1       0.51      0.54      0.53        48
          2       0.39      0.53      0.45        81
          3       0.00      0.00      0.00         0
          4       0.39      0.57      0.46        75
          5       0.94      0.82      0.88       765
          6       0.71      0.83      0.77        12

avg / total       0.82      0.76      0.79       998

